# Assignment: Non-Boltzmann Sampling

---

### 🧪 **Objective:**
We will use the **Metropolis Monte Carlo (MC) method** with **non-Boltzmann sampling** to **compute the energy barrier** for a **Cu adatom** migrating on a **Cu(111) surface** from an **FCC adsorption site** to an **HCP adsorption site**.

---

### 🖼️ **System Setup:**
<img src="Cu_adatom.png" width="500">

- **Substrate:** Cu(111) surface with Cu adatom
- **Initial Position:** **FCC adsorption site**

---

### ⚙️ **Energy Computation:**
- **Simulation Engine:** **LAMMPS**
- **Potential:** **EAM (Embedded Atom Method)** for Cu
- **Units:**
  - **Energy:** eV
  - **Distances:** Å

---

### 🔍 **Averaged quantities:**
1. **Enthalpy** of different configurations.
2. **Free Energy** derived from **MC statistics**.
3. **CV** value.

---

### 🎯 **Key Challenge:**
To enable the adatom to move from its **initial position** to the **HCP site**, we will apply an **"umbrella" energy potential** to **guide the sampling**.

---



In [ ]:
# Import libraries and read geometry files of the Cu clusters
import numpy as np
from scipy.constants import physical_constants
from ase import Atoms,Atom
from ase.io import read,write
from ase.build import fcc111
import nglview as nv
import time
import matplotlib.pyplot as plt
from ase.calculators.lammpsrun import LAMMPS
from IPython.display import Audio
from scipy.interpolate import CubicSpline
import ipywidgets as ipw

In [ ]:
def format_number(x, _):
    if x >= 1e6:
        return f'{x/1e6:.1f}M'
    elif x >= 1e4:
        return f'{x/1e3:.1f}k'
    else:
        return str(int(x))

In [ ]:
output = ipw.Output()

In [ ]:
#Functions to visualize the geometries
def view_structure(structure):
    """
    Use the ASE library to view an atoms object.

    Parameters
    ----------

    structure: Atoms object

    Returns
    -------

    NGLWidget with GUI: object to be viewed
    
    """
    #t1=make_supercell(structure,[[3,0,0],[0,3,0],[0,0,3]])
    t = nv.ASEStructure(structure)
    w = nv.NGLWidget(t, gui=True)
    w.add_unitcell()
    w.add_spacefill()
    return w

def view_trajectory(trajectory):
    t2 = nv.ASETrajectory(trajectory)
    w2 = nv.NGLWidget(t2, gui=True)
    #w2.add_unitcell()
    w2.add_ball_and_stick()
    return w2

In [ ]:
def distance1D(structure,id1,id2,L):
    """computes the distance along y  between two atoms"""
    return (structure[id1].y - structure[id2].y) %L

## Construct Cu(111) slabs with a Cu adatom in 4 different positions 

In [ ]:
slab = fcc111('Cu', size=(4, 4, 3), vacuum=15.0, a=3.61,orthogonal=True)
slab.pbc=(True,True,True)
dz=slab[16].z-slab[0].z
fcc = slab.copy() + Atom('Cu',position=slab[6].position + np.array([0,0,3*dz -0.2]))
ts = slab.copy() + Atom('Cu',position=(slab[37].position + slab[38].position )/2 +np.array([0,0,dz+0.05]))
hcp = slab.copy() + Atom('Cu',position=slab[18].position + np.array([0,0,2*dz -0.2]))
top = slab.copy() + Atom('Cu',position=slab[34].position + np.array([0,0,dz +0.3]))
natoms=len(fcc)

In [ ]:
#view_structure(ts)
#fcc.write("fcc.xyz")

## We define a collective variable CV as the y distance betwee the adatom and the surface atom #42. When CV = 0 the adatom is in "top" position.

In [ ]:
L = np.abs(fcc[34].y -fcc[42].y)
dfcc = distance1D(fcc,48, 34,L=L)
dts = distance1D(ts,48, 34,L=L)
dhcp = distance1D(hcp,48, 34,L=L)
print(f"distance at FCC: {dfcc:.3} TS: {dts:.3} HCP: {dhcp:.3} L: {L:.3}")

In [ ]:
calculator_does_not_work = False
if calculator_does_not_work:
    workdir = !pwd
    workdir = workdir[0]
    inp=f"""# (written by ASE)
clear
atom_style atomic
units metal
boundary p p p
atom_modify sort 0 0.0

## interactions
pair_style eam
read_data data_lammps_cu
pair_coeff * * Cu_u3.eam
mass 1 63.546

## run
fix fix_nve all nve
dump dump_all all custom 1 {workdir}/trj_lammps.bin id type x y z vx vy vz fx fy fz
thermo_style custom step temp press cpu pxx pyy pzz pxy pxz pyz ke pe etotal vol lx ly lz atoms
thermo_modify flush yes format float %23.16g
thermo 1
run 0
print "__end_of_ase_invoked_calculation__"
log /dev/stdout"""
    with open(f"{workdir}/lammps_cu.in", "w") as text_file:
        text_file.write(inp)


    data_lammps0=f"""data_lammps={workdir}/data_lammps_cu (written by ASE)

{natoms}       atoms
1  atom types
0.0      {fcc.cell[0][0]}  xlo xhi
0.0      {fcc.cell[1][1]}  ylo yhi
0.0      {fcc.cell[2][2]}  zlo zhi


"""

    # In case ase.calculator does not work
    def lammps_positions(atoms):
        """Function to write the postions of an ASE Atoms object in the lammps input
        -iput: ASE Atoms object
        -output: string with positions
        """
        #lmpid={'C':1,'H':2}
        positions="""Atoms
        
"""
        for i,atom in enumerate(atoms):
            positions+=f"{i+1:4} 1 {atom.x:14.8} {atom.y:14.8} {atom.z:14.8} \n"
        return positions


    def get_energy(atoms):  
        """Due to a bug in the lammps ASE calculator we write an ad-hoc function to run a lammps calculation
        to compute the energy of a configuration
        -iput ASE Atoms object
        -outpt Enthalpy in eV
        """
        data_lammps = data_lammps0 + lammps_positions(atoms)
        with open(f"{workdir}/data_lammps_cu", "w") as text_file:
            text_file.write(data_lammps)
        out = !lmp_serial < ./lammps_cu.in  | grep -A 100000 "Step          Temp          Press"
        # parsing of lammps.log (out) to get the energy of the configuration
        results = out[1].split()
        return float(results[11])
else:
    parameters = {'pair_style': 'eam',
                'pair_coeff': ['* * Cu_u3.eam']}
    files = ['Cu_u3.eam']
    lammps = LAMMPS(files=files, **parameters)

    def get_energy(asegeo):
        asegeo.calc = lammps
        return asegeo.get_potential_energy()    

## Familiarizing with the system: energy of "FCC" "HCP",  "BRIDGE" and "TOP" configurations (not optimized)

In [ ]:
# testing the get_energy function
start_time = time.time()
efcc = get_energy(fcc)
ets=get_energy(ts) 
ehcp = get_energy(hcp)
etop = get_energy(top)
end_time = time.time()
ets-efcc
execution_time = end_time - start_time
print(f"""Energy FCC (eV) {efcc:.3f} Energy TS (eV) {ets:.3f} Energy HCP (eV) {ehcp:.3f} Energy top: {etop:.3f}
DE TS - FCC (eV) {ets-efcc:.6} 
DE TS - HCP (eV) {ets-ehcp:.6}""")
print(f"Execution time {execution_time:.3f} s")

## Initializing histograms

In [ ]:
nintervals=200
hist, bin_edges = np.histogram([1.0,2.0,3.0], bins=nintervals, range=(0, L))

In [ ]:
def find_minimum_energy(L, y_values, nz_intervals, z_min=-0.2, z_max=1.0):
    """
    Computes the minimum energy and corresponding z value for each y in the specified interval.
    
    :param get_energy: Function that computes energy for the given 'ase_geo' object.
    :param L: The upper limit for the y interval (0, L).
    :param ny_intervals: Number of intervals in the y range.
    :param nz_intervals: Number of intervals in the z range.
    :param z_min: The lower limit for the z interval (-0.3 by default).
    :param z_max: The upper limit for the z interval (1.0 by default).
    :return: A list of tuples (y, min_energy, min_z) for each y value.
    """
    
    results = []

    # Define y and z ranges
    z_values = np.linspace(z_min, z_max, nz_intervals)

    # Loop through each y value
    for y in y_values:
        energies = []
        
        for z in z_values:
            # Create an ASE geometry object (e.g., FCC 111 surface)
            geo = fcc.copy()
            
            # Update the position of atom index 48
            geo[48].z = z
            geo[48].y = y
            
            # Calculate energy 
            energy = get_energy(geo)
            energies.append(energy)

        min_energy = min(energies)  # Find the minimum energy
        min_z = z_values[np.argmin(energies)]  # Corresponding z value
        results.append((y, min_energy, min_z))
    
    return results

## Initial Bias Potential Setup

- We initially set the **bias potential** (`bias_energies`) as the **energy `E(y)`** of the **adatom** when it **moves along the y-axis** from **0** (**top position**) to **L** (**equivalent top position**).
- During this process, the **z-coordinate** is **kept fixed** at a **specific value**.


In [ ]:
def compute_energies_at_z(y_values, z, georef):
    """
    Computes the energy for a given z value and a list of y values.
    
    :param y_values: List of y values to evaluate.
    :param z: The specific z value for which energies are computed.
    :param fcc: The ASE geometry object to use as a template.
    :param get_energy: Function that computes energy for the given 'ase_geo' object.
    :return: A list of energies corresponding to the input y values.
    """
    
    energies = []

    # Loop through each y value
    for y in y_values:
        # Create an ASE geometry object (e.g., FCC surface)
        geo = georef.copy()
        
        # Update the position of atom index 48
        geo[48].z = z
        geo[48].y = y
        
        # Calculate energy
        energy = get_energy(geo)
        energies.append(energy)
    
    return np.array(energies)

# Simulated input data
y_values = bin_edges
z = fcc[48].z + 0.25  # Specific z value for which to compute energies

# Compute energies
bias_energies = compute_energies_at_z(y_values, z, fcc) -efcc




## Plot the energy profile (plot #1)

In [ ]:
# Plot energy vs y values
plt.figure(figsize=(8, 6))
plt.plot(y_values, bias_energies, marker='o', linestyle='-', color='blue', label=f'Energy at z = {z}')
plt.xlabel('y value')
plt.ylabel('Energy')
plt.title('Energy vs y values at specific z')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
#min_energy_results = find_minimum_energy(L, bin_edges, 50, z_min=fcc[48].z +0., z_max=fcc[48].z+0.4)

## Minimum energy path 

In [ ]:
#y_values = [y for y, min_energy, min_z in min_energy_results]
#energy_values = [min_energy -efcc for y, min_energy, min_z in min_energy_results]
#z_values = [min_z - fcc[48].z for y, min_energy, min_z in min_energy_results]

# Plot 1: Energy vs y
#plt.figure(figsize=(8, 6))
#plt.plot(y_values, energy_values , marker='o', linestyle='-', color='blue', label='Energy')
#plt.xlabel('y value')
#plt.ylabel('Minimum Energy')
#plt.title('Energy vs y')
#plt.grid(True)
#plt.legend()
#plt.show()



In [ ]:
# Plot 2: z vs y
#plt.figure(figsize=(8, 6))
#plt.plot(y_values, z_values, marker='o', linestyle='-', color='green', label='z value')
#plt.xlabel('y value')
#plt.ylabel('z value at Minimum Energy')
#plt.title('z vs y')
#plt.grid(True)
#plt.legend()
#plt.show()

## functions to create the bias potential with splines

In [ ]:
def subdivide_range(L, nsegments, y1, y2, y3, e1, e2):
    # Generate y values for each bin center
    y_values = np.linspace(0, L, nsegments)
    values = np.zeros(nsegments)
    
    # Set values to 0 from 0 to y1
    mask_y1 = (y_values >= 0) & (y_values < y1)
    values[mask_y1] = 0
    
    # Increase linearly from 0 to e1 between y1 and y2
    mask_y2 = (y_values >= y1) & (y_values < y2)
    values[mask_y2] = np.linspace(0, e1, mask_y2.sum())
    
    # Decrease linearly from e1 to e1-delta between y2 and y3
    mask_y3 = (y_values >= y2) & (y_values < y3)
    values[mask_y3] = np.linspace(e1, e1-e2, mask_y3.sum())
    
    # Decrease linearly from e1-delta to 0 between y3 and L
    mask_y4 = (y_values >= y3) & (y_values <= L)
    values[mask_y4] = np.linspace(e1-e2, 0, mask_y4.sum())
    
    return y_values, values

def generate_spline(L, bin_edges, points):
    """
    Generates a cubic spline over the range [0, L] using given bin edges and points.

    Parameters:
    - L (float): The upper bound of the range.
    - bin_edges (list of float): The discretized edges of the range [0, L].
    - points (list of tuples): The points to interpolate, e.g. [(0,1),(1.2,0.3),(1.5,0.2),(L,1)].

    Returns:
    - spline_values (np.ndarray): The spline values at the bin edges.
    """
    # Extract x and y coordinates from the points
    x_points, y_points = zip(*points)

    # Create a cubic spline interpolation
    spline = CubicSpline(x_points, y_points, bc_type='natural')

    # Evaluate the spline at the bin edges
    spline_values = spline(bin_edges)

    return spline_values

def assign_to_bin(value, bin_edges):
    """
    Assign a float value to the correct bin based on the histogram bin edges.
    
    Args:
        value (float): The value to assign to a bin.
        bin_edges (np.ndarray): The array of bin edges from np.histogram.
        
    Returns:
        int: The index of the bin to which the value belongs.
    """
    # Ensure the value is within the range
    if value < bin_edges[0] or value > bin_edges[-1]:
        raise ValueError(f"Value {value} is out of the histogram range [{bin_edges[0]}, {bin_edges[-1]}]")
    
    # Find the correct bin index using np.digitize
    bin_index = np.digitize(value, bin_edges) - 1
    
    # Handle edge case where value is exactly the last edge
    if bin_index == len(bin_edges) - 1:
        bin_index -= 1
    
    return bin_index

## bias potential from splines

In [ ]:
from_splines = False
plot_title = 'Energy vs y values at specific z. Same as plot #1'
if from_splines:
    plot_title = 'Segmented Function with Custom Linear Interpolation'
    bias_energies = generate_spline(L, bin_edges, [(0,0.65),(0.5,0.5),(dhcp,0.15),(dts,0.25),(dfcc,0.15),(L-0.5,0.5),(L,0.65)])

## bias potential (plot #2 will be identical to #1 if from_splines = False)

In [ ]:
# Plotting the result to visualize the function
plt.figure(figsize=(8, 4))
plt.plot(bin_edges, bias_energies, '-o', markersize=3)
plt.xlabel('y')
plt.ylabel('Value')
plt.title(plot_title)
plt.grid(True)
plt.show()

# Question 1 (5 points)

## Complete the cell below to implement the correct Metropolis Monte Carlo (MC) algorithm.
- **Initial Test:** Run the MC simulation with a **small number of steps** (e.g., **100k**) to verify if the **bias potential** we defined **samples the bridge position** correctly.

- **Adjust Bias Potential:** 
  1. **Go back** in the notebook and **set `from_splines = True`** to **adjust the bias potential**.
  2. The **suggested parameters** should already lead to **better sampling**.

- **Main Simulation:** Now **run the MC loop** with approximately **600k steps**.

---

### 💡 **Additional Questions:**
1. **What happens to the acceptance rate** if we decide to **move all atoms** at each step instead of **just the adatom**?
2. **How would you design such moves** to ensure:
   - The **system remains stable** and **undamaged**?
   - The **acceptance rate** remains **reasonable**?


### answer: 

## MC loop
#### In the following cell, core of the MC procedure, we use "new" and "previous" to be able to store quantities in case MC moves are rejected. We store for all configurations: the distance betweem atoms 48 and 34, the energy. We store, for animations, configurations every iwrite steps. For simplicity we move only the adatom. An histogram for teh different values of CV is also updated every iwrite steps

In [ ]:
display(output)
kb=physical_constants['Boltzmann constant in eV/K'][0]
enthalpies = np.zeros(nintervals)
T = 170 #temperature in K
beta =1.0/(kb*T)

# Initial values for the MC trajectory
previous_geo = fcc
previous_cv = dfcc
previous_index = assign_to_bin(previous_cv, bin_edges)
previous_enthalpy = get_energy(previous_geo)
bias_energy = bias_energies[previous_index]
previous_ene = previous_enthalpy - bias_energy
#print(previous_enthalpy,previous_ene)

iwrite=10000
niter=600000 #you can reduce this to 600000 and it should take less than 1 hour
start = time.time()
geometries=[previous_geo]
distances = []
distances.append(previous_cv)
enthalpies[previous_index] += previous_enthalpy 
try:
    rm_traj = !rm traj_cu.xyz
except:
    pass

naccepted = 0
minz=fcc[48].z
maxz=fcc[48].z+0.5
with output:
    #output.clear_output()
    print("MC loop started.")
    for i in range(niter):
        new_geo = previous_geo.copy()
        cu_y = np.random.uniform(0, L)
        cu_z = np.random.uniform(minz, maxz)
        new_cv = cu_y
        new_geo[48].y = cu_y #48 is the index of the Cu adatom
        new_geo[48].z = cu_z
        new_enthalpy =  get_energy(new_geo)
        new_index = assign_to_bin(cu_y, bin_edges)
        bias_energy = bias_energies[new_index]
        new_ene = new_enthalpy - bias_energy
        #print(cu_y,cu_z,new_enthalpy,new_ene,np.exp(-beta * (new_ene - previous_ene)))

        # write here the Metropolis condition for acceptance
        rho = np.random.random() #random number in [0,1)\n",
        if :#..... enter here the  Metropolis condition:
            # update old variables with new accepted values
            naccepted+=1
            previous_ene = new_ene
            previous_enthalpy = new_enthalpy
            previous_geo = new_geo.copy()
            previous_cv = new_cv
            previous_index = new_index

        # is it correct to update statistics like this or the following two lines should be inside the if?    
        distances.append(previous_cv)
        enthalpies[previous_index] += previous_enthalpy        

        if not np.mod(i,iwrite) and i>0:
            output.clear_output()
            geometries.append(previous_geo)
            print(f"MC loop started. Done {i} steps in {int(time.time() -start)} seconds. Acceptance rate {int(1.*naccepted/i *100)} % ")
            hist_tmp, bin_edges_tmp = np.histogram(distances, bins=20, range=(0,L), density=None, weights=None)
            plt.figure(figsize=(8, 6))
            plt.bar(bin_edges_tmp[:-1], hist_tmp, width=np.diff(bin_edges_tmp), edgecolor='black', align='edge', color='skyblue')

            # Add labels and title
            plt.xlabel('Distance (bins)')
            plt.ylabel('Frequency')
            plt.title('Histogram of Distances')
            plt.grid(axis='y', linestyle='--', alpha=0.7)

            # Add value labels on top of bars
            for x, y in zip(bin_edges_tmp[:-1], hist_tmp):
                plt.text(x + 0.1, y + 1,format_number(y, None), ha='center', va='bottom', fontsize=6)

            plt.tight_layout()
            plt.show()


    end = time.time()
    print("MC loop completed.")
    print(f"Total execution time {end - start} s") 
    print(f"Acceptance rate {int(1.*naccepted/niter *100)} %")

In [ ]:
view_trajectory(geometries) #the structure is upside down, rotate it to see the adatom

In [ ]:
if False:
    for atoms in geometries:
        #atoms.info={'energy':energies_array[i]}
        atoms.write('traj_cu.xyz',format="extxyz",append=True)

In [ ]:
hist, bin_edges = np.histogram(distances, bins=nintervals, range=(0,L), density=None, weights=None)

In [ ]:
hist

In [ ]:
histogram0 = hist # np.flip(hist)

In [ ]:
histogram0

In [ ]:
len(histogram0)

In [ ]:
shift = np.zeros(len(histogram0))
if np.any(histogram0 == 0):
    shift.fill(0.1)
shifted_histogram = 1.0*histogram0 + shift

In [ ]:
Q0=shifted_histogram/np.sum(shifted_histogram)

In [ ]:
A0=-np.log(Q0)/beta

# Question 2 (points 2)
## explain in few words the formula above (make a reference to the lecture)

### answer:

## Plot#2

In [ ]:
fig, ax = plt.subplots()
#ax.plot(np.flip(bin_edges)[0:nintervals],A0)
ax.plot(bin_edges[0:nintervals],A0)
ax.set_xlabel("CV (Å)")
ax.set_ylabel("Not re-weighted free energy (eV)")
#ax.set_ylim(0,0.1)

In [ ]:
#Q = Q0 * np.exp(-beta*bias_energies)
Q = Q0 * np.exp(-beta*bias_energies[0:nintervals])

# Question 3 (points 2)
## explain in few words the formula above (make a reference to the lecture)

### answer:

In [ ]:
# correct free energy after re-weighting
A = -np.log(Q)/beta

## Plot#3

In [ ]:
# in the following plots I cut the regions that were not explored by the adatom
fig, ax = plt.subplots()
ax.plot(bin_edges[0:nintervals],A)
ax.set_xlabel("CV (Å)")
ax.set_ylabel("Re-weighted free energy (eV)")
#ax.set_ylim(0,0.1)
#plt.plot(np.flip(histogram[1])[0:200],A)

# Question 4 (points 1)
## Why is the result "noisy" ?

### answer:

## Enthalpy

In [ ]:
#enthalpies

In [ ]:
max_ene=etop - efcc
enthalpies_avg = [
    max_ene if nvisited == 0 else totene / nvisited -efcc
    for nvisited, totene in zip(histogram0, enthalpies)
]

## Plot#4

In [ ]:
fig, ax = plt.subplots()
ax.plot(bin_edges[0:nintervals],enthalpies_avg)
ax.set_xlabel("CV (Å)")
ax.set_ylabel("Averaged enthalpies(eV)")
ax.set_ylim(0,0.55)
#plt.plot(np.flip(histogram[1])[0:200],enthalpies_avg)